In [1]:
from openfermion import (
    QubitOperator as Q,
    commutator,
    anticommutator,
    get_sparse_operator as gso,
    get_ground_state as ggs,
    variance,
    bravyi_kitaev,
    FermionOperator
)
import numpy as np
from fermi_util import get_commutator
from algo_utils import bliss_three_body, bliss_three_body_cheaper
from main_utils_partitioning import N_QUBITS, copy_hamiltonian, sorted_insertion_decomposition
import pickle

In [2]:
def abs_of_dict_value(x):
    return np.abs(x[1])

def ferm_to_qubit(H: FermionOperator):
    Hqub = bravyi_kitaev(H)
    Hqub -= Hqub.constant
    Hqub.compress()
    Hqub.terms = dict(
    sorted(Hqub.terms.items(), key=abs_of_dict_value, reverse=True))
    return Hqub

In [3]:
moltag = 'h2'
filename = f'commutator_store/{moltag}_commutator.pkl'
with open(filename, 'rb') as f:
    H = pickle.load(f)
    
filename2 = f'commutator_store/{moltag}_commutator_blissed.pkl'
with open(filename2, 'rb') as f:
    H_bliss = pickle.load(f)

In [4]:
print(H)

0.3935811669709405 [1^ 0^ 1 0] +
1.0701145997683443 [1^ 0^ 3 2] +
-0.3935811669709405 [2^ 1^ 0^ 2 1 0] +
-0.3935811669709405 [3^ 1^ 0^ 3 1 0] +
0.3935811669709405 [3^ 2^ 0^ 3 2 0] +
1.0701145997683443 [3^ 2^ 1 0] +
0.3935811669709405 [3^ 2^ 1^ 3 2 1] +
-0.3935811669709405 [3^ 2^ 3 2]


In [5]:
H_q = ferm_to_qubit(H)
H_bliss_q = ferm_to_qubit(H_bliss)

In [6]:
def commutator_variance(H: FermionOperator, decomp, num_modes):
    """
    Computes the variance of the [H, G] - K.
    :param H: The Hamiltonian to compute the variance of.
    :param decomp: The decomposition of the Hamiltonian.
    :param G: The fermion operator to define the gradient
    :param K: The Killer operator applied to the whole [H, G]
    :param N: The number of sites
    :return: The variance metric
    """
    psi = ggs(gso(H, num_modes))[1]
    vars = np.zeros(len(decomp), dtype=np.complex128)
    for i, frag in enumerate(decomp):
        vars[i] = variance(gso(frag, num_modes), psi)
    return np.sum((vars) ** (1 / 2)) ** 2

In [7]:
from qwc_decomposition import qwc_decomposition

In [8]:
methodtag = 'ac'
N = 4
decomp = qwc_decomposition(H_q)
print("Original Decomposition Complete")
blissed_decomp = qwc_decomposition(H_bliss_q)
print("Blissed Decomposition Complete")
var = commutator_variance(H_q, decomp, N)

blissed_vars = commutator_variance(H_bliss_q, blissed_decomp, N)

print(f"Original variance: {var}")
print(f"Blissed variance: {blissed_vars}")

Original Decomposition Complete
Blissed Decomposition Complete
Original variance: (0.5457938873939997+6.938893903907228e-18j)
Blissed variance: (0.545793887393999-2.7393585667575274e-17j)
